In [1]:
pip install -r ../../requirements.txt

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [2]:
# Export model
def export_onnx(model, data_tensor_array, model_loc):
  circuit = model()

  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

  # print(device)

  circuit.to(device)

  # Flips the neural net into inference mode
  circuit.eval()
  input_names = []
  dynamic_axes = {}

  data_tensor_tuple = ()
  for i in range(len(data_tensor_array)):
    data_tensor_tuple += (data_tensor_array[i],)
    input_index = "input"+str(i+1)
    input_names.append(input_index)
    dynamic_axes[input_index] = {0 : 'batch_size'}
  dynamic_axes["output"] = {0 : 'batch_size'}

  # Export the model
  torch.onnx.export(circuit,               # model being run
                      data_tensor_tuple,                   # model input (or a tuple for multiple inputs)
                      model_loc,            # where to save the model (can be a file or file-like object)
                      export_params=True,        # store the trained parameter weights inside the model file
                      opset_version=11,          # the ONNX version to export the model to
                      do_constant_folding=True,  # whether to execute constant folding for optimization
                      input_names = input_names,   # the model's input names
                      output_names = ['output'], # the model's output names
                      dynamic_axes=dynamic_axes)

In [3]:
# mode is either "accuracy" or "resources"

def gen_settings(comb_data_path, onnx_filename, scale, mode, settings_filename):
  print("==== Generate & Calibrate Setting ====")
  # Set input to be Poseidon Hash, and param of computation graph to be public
  # Poseidon is not homomorphic additive, maybe consider Pedersens or Dory commitment.
  gip_run_args = ezkl.PyRunArgs()
  gip_run_args.input_visibility = "hashed"  # matrix and generalized inverse commitments
  gip_run_args.output_visibility = "public"   # no parameters used
  gip_run_args.param_visibility = "private" # should be Tensor(True)--> to enforce arbitrary data in w

 # generate settings
  ezkl.gen_settings(onnx_filename, settings_filename, py_run_args=gip_run_args)
  if scale =="default":
    ezkl.calibrate_settings(
    comb_data_path, onnx_filename, settings_filename, mode)
  else:
    ezkl.calibrate_settings(
    comb_data_path, onnx_filename, settings_filename, mode, scales = scale)

  assert os.path.exists(settings_filename)
  assert os.path.exists(comb_data_path)
  assert os.path.exists(onnx_filename)
  f_setting = open(settings_filename, "r")
  print("scale: ", scale)
  print("setting: ", f_setting.read())


In [4]:
def verifier_init(verifier_model, verifier_model_path, verifier_compiled_model_path, dummy_data_path_array, settings_path, srs_path, pk_path, vk_path, scale, mode):

  # load data from dummy_data_path_array into dummy_data_array
  dummy_data_tensor_array = []
  comb_dummy_data = []
  for path in dummy_data_path_array:
    dummy_data = np.array(json.loads(open(path, "r").read())["input_data"][0])
    # print("dumm: ", dummy_data)
    dummy_data_tensor_array.append(torch.reshape(torch.tensor(dummy_data), (1, len(dummy_data),1 )))
    comb_dummy_data.append(dummy_data.tolist())
  # export onnx file
  export_onnx(verifier_model,dummy_data_tensor_array, verifier_model_path)

  comb_dummy_data_path = os.path.join('generated/comb_dummy_data.json')
  # Serialize data into file:
  json.dump(dict(input_data = comb_dummy_data), open(comb_dummy_data_path, 'w' ))

  # gen + calibrate setting
  gen_settings(comb_dummy_data_path, verifier_model_path, scale, mode, settings_path)

  # compile circuit
  res = ezkl.compile_circuit(verifier_model_path, verifier_compiled_model_path, settings_path)
  assert res == True

  # srs path
  res = ezkl.get_srs(srs_path, settings_path)

  # setupt vk, pk param for use..... prover can use same pk or can init their own!
  print("==== setting up ezkl ====")
  start_time = time.time()
  res = ezkl.setup(
        verifier_compiled_model_path,
        vk_path,
        pk_path,
        srs_path)
  end_time = time.time()
  time_setup = end_time -start_time
  print(f"Time setup: {time_setup} seconds")

  assert res == True
  assert os.path.isfile(vk_path)
  assert os.path.isfile(pk_path)
  assert os.path.isfile(settings_path)

In [5]:
def prover_gen(prover_model, data_path_array, witness_path, prover_model_path, prover_compiled_model_path, settings_path, proof_path):
  # load data from data_path
  # data = json.loads(open(data_path, "r").read())["input_data"][0]
  # data_tensor = torch.reshape(torch.tensor(data), (1, len(data),1 ))


  data_tensor_array = []
  comb_data = []
  for path in data_path_array:
    data = np.array(json.loads(open(path, "r").read())["input_data"][0])
    # print("dumm: ", dummy_data)
    data_tensor_array.append(torch.reshape(torch.tensor(data), (1, len(data),1 )))
    comb_data.append(data.tolist())

  # export onnx file
  export_onnx(prover_model, data_tensor_array, prover_model_path)

  comb_data_path = os.path.join('generated/comb_data.json')
  # Serialize data into file:
  json.dump(dict(input_data = comb_data), open(comb_data_path, 'w' ))

  res = ezkl.compile_circuit(prover_model_path, prover_compiled_model_path, settings_path)
  assert res == True
  # now generate the witness file
  print('==== Generating Witness ====')
  witness = ezkl.gen_witness(comb_data_path, prover_compiled_model_path, witness_path)
  assert os.path.isfile(witness_path)
  # print(witness["outputs"])
  settings = json.load(open(settings_path))
  output_scale = settings['model_output_scales']
  print("witness boolean: ", ezkl.vecu64_to_float(witness['outputs'][0][0], output_scale[0]))
  for i in range(len(witness['outputs'][1])):
    print("witness result", i+1,":", ezkl.vecu64_to_float(witness['outputs'][1][i], output_scale[1]))

  # GENERATE A PROOF
  print("==== Generating Proof ====")
  start_time = time.time()
  res = ezkl.prove(
        witness_path,
        prover_compiled_model_path,
        pk_path,
        proof_path,
        srs_path,
        "single",
    )

  print("proof: " ,res)
  end_time = time.time()
  time_gen_prf = end_time -start_time
  print(f"Time gen prf: {time_gen_prf} seconds")
  assert os.path.isfile(proof_path)

In [6]:
def verifier_verify(proof_path, settings_path, vk_path, srs_path):
  # enforce boolean statement to be true
  settings = json.load(open(settings_path))
  output_scale = settings['model_output_scales']

  proof = json.load(open(proof_path))
  num_inputs = len(settings['model_input_scales'])
  print("num_inputs: ", num_inputs)
  proof["instances"][0][num_inputs] = ezkl.float_to_vecu64(1.0, output_scale[0])
  json.dump(proof, open(proof_path, 'w'))

  print("prf instances: ", proof['instances'])

  print("proof boolean: ", ezkl.vecu64_to_float(proof['instances'][0][num_inputs], output_scale[0]))
  for i in range(num_inputs+1, len(proof['instances'][0])):
    print("proof result",i-num_inputs,":", ezkl.vecu64_to_float(proof['instances'][0][i], output_scale[1]))


  res = ezkl.verify(
        proof_path,
        settings_path,
        vk_path,
        srs_path,
    )

  assert res == True
  print("verified")

In [7]:
# init path
os.makedirs(os.path.dirname('generated/'), exist_ok=True)
verifier_model_path = os.path.join('generated/verifier.onnx')
prover_model_path = os.path.join('generated/prover.onnx')
verifier_compiled_model_path = os.path.join('generated/verifier.compiled')
prover_compiled_model_path = os.path.join('generated/prover.compiled')
pk_path = os.path.join('generated/test.pk')
vk_path = os.path.join('generated/test.vk')
proof_path = os.path.join('generated/test.pf')
settings_path = os.path.join('generated/settings.json')
srs_path = os.path.join('generated/kzg.srs')
witness_path = os.path.join('generated/witness.json')

=======================  ZK-STATS FLOW =======================

In [10]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('generated/dummy_data.json')

f_raw_input = open(data_path, "r")
data = json.loads(f_raw_input.read())["input_data"][0]
data_tensor = torch.reshape(torch.tensor(data),(1, len(data), 1))

#  dummy data for data consumer: make the bound approx same as real data
dummy_data = np.random.uniform(min(data), max(data), len(data))
json.dump({"input_data":[dummy_data.tolist()]}, open(dummy_data_path, 'w'))

dummy_data_tensor = torch.reshape(torch.tensor(dummy_data), (1, len(dummy_data),1 ))
dummy_theory_output = torch.exp(torch.mean(torch.log(dummy_data_tensor)))



In [14]:
print(dummy_theory_output)

tensor(46.1673, dtype=torch.float64)


In [20]:
# Verifier/ data consumer side:
class verifier_model(nn.Module):
    def __init__(self):
        super(verifier_model, self).__init__()
        # w represents mean in this case
        self.w = nn.Parameter(data = dummy_theory_output, requires_grad = False)

    def forward(self,X):
        # some expression of tolerance to error in the inference
        return (torch.abs((torch.log(self.w)*X.size()[1])-torch.sum(torch.log(X)))<0.01*(torch.log(self.w)*X.size()[1]), self.w)

verifier_init(verifier_model, verifier_model_path, verifier_compiled_model_path, [dummy_data_path], settings_path, srs_path, pk_path, vk_path,[0], "resources")

x size:  tensor(300)
==== Generate & Calibrate Setting ====
scale:  [0]
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":0,"param_scale":0,"scale_rebase_multiplier":10,"lookup_range":[-36,174],"logrows":14,"num_inner_cols":1,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private"},"num_rows":14432,"total_assignments":906,"total_const_size":0,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,0],"model_input_scales":[0],"module_sizes":{"kzg":[],"poseidon":[14432,[1]],"elgamal":[0,[0]]},"required_lookups":["Abs",{"Ln":{"scale":1.0}},{"GreaterThan":{"a":0.0}}],"check_mode":"UNSAFE","version":"5.0.8","num_blinding_factors":null}


spawning module 0
spawning module 2


==== setting up ezkl ====


spawning module 0
spawning module 2


Time setup: 1.4332208633422852 seconds


In [21]:
# Prover/ data owner side
theory_output = torch.mean(data_tensor)
class prover_model(nn.Module):
    def __init__(self):
        super(prover_model, self).__init__()
        # w represents mean in this case
        self.w = nn.Parameter(data = theory_output, requires_grad = False)

    def forward(self,X):
        # some expression of tolerance to error in the inference
        return (torch.abs((torch.log(self.w)*X.size()[1])-torch.sum(torch.log(X)))<0.01*(torch.log(self.w)*X.size()[1]), self.w)
prover_gen(prover_model, [data_path], witness_path, prover_model_path, prover_compiled_model_path, settings_path, proof_path)

spawning module 0


==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 50.0
==== Generating Proof ====


spawning module 2


proof:  {'instances': [[[10512373747352303962, 11798585516934984832, 13421675179368312123, 2200257403316998104], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [18373208119716085496, 9977853902793730609, 17093442942004944878, 1769661249493325049]]], 'proof': '1fce32c01563627db5c00bc2ecbd9c60430802494110aaead4a76e1b3ff83e2501ec5deefd663536b84ed46cdc20124a136f4c5daa80f149a4fd9cf5f5d8bfb60767218d6f8bc2a594fd7df932f0bfcd6e81aa9bffdc5086cda331c694579e0210ee0904d2e539a9b6e568a37e77c33e2d3439819bca6071f1721c50481d8a79083a7457e6a143b031e29dd5fd386919a3447caef0eba4fa7e7d38e3e686ec281a14878b6d6b811d307ce103baa55ef2d4d7862c9d82d34d2db349a5ce68fc5906e8bfe067d00d9d55592a2e0287894a4224452f549f020a3c5da94ba70bfbc406bee7f5dc1a244d3791fd353f7494cd7e4ba5d0dfb26113554de360312a7f9a11c828114ceafccc11eac18736b6a24791fa717d636315ec44aa02ccb33c74d40659dd626491a623a8447c67b09ab06eede5d01e845ed12c8bd5c399a42c4b5f114815129e36d1aae69bba2bff0f2c2e3ddef2dd141d050cda9746388c26

In [22]:
# Verifier verifies
verifier_verify(proof_path, settings_path, vk_path, srs_path)
print("theory mean: ", theory_output)

num_inputs:  1
prf instances:  [[[10512373747352303962, 11798585516934984832, 13421675179368312123, 2200257403316998104], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [18373208119716085496, 9977853902793730609, 17093442942004944878, 1769661249493325049]]]
proof boolean:  1.0
proof result 1 : 50.0
verified
theory mean:  tensor(49.9700)
